In [28]:
import tensorflow as tf
import keras
from keras import layers
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import glob
import os



In [29]:
(train_images,  train_labels), (_, _) = tf.keras.datasets.mnist.load_data()

In [30]:
train_images = np.expand_dims(train_images,axis=-1)

In [31]:
train_images.shape

(60000, 28, 28, 1)

In [32]:
train_images.dtype

dtype('uint8')

In [33]:
train_images = train_images.astype('float32')

In [34]:
train_images = (train_images - 127.5)/127.5

In [35]:
BATCH_SIZE = 256
BUFFER_SIZE = 60000

In [36]:
datasets = tf.data .Dataset.from_tensor_slices(train_images)

In [37]:
datasets = datasets.shuffle(BUFFER_SIZE).batch(BATCH_SIZE)

In [38]:
datasets

<BatchDataset shapes: (None, 28, 28, 1), types: tf.float32>

In [39]:
def generator_model():
    model = keras.Sequential()
    model.add(layers.Dense(256,input_shape=(100,),use_bias=False))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())
    
    model.add(layers.Dense(512,use_bias=False))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())
    
    model.add(layers.Dense(28*28*1,use_bias='False',activation='tanh'))
    model.add(layers.BatchNormalization())

    model.add(layers.Reshape((28,28,1)))
    return model

In [40]:
def discriminator_model():
    model = keras.Sequential()
    model.add(layers.Flatten())
    
    model.add(layers.Dense(512,use_bias=False))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())
    
    model.add(layers.Dense(256,use_bias=False))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())
    
    model.add(layers.Dense(1))
    return model
    

In [41]:
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)

In [42]:
def discriminator_loss(real_out,fake_out):
    read_loss = cross_entropy(tf.ones_like(real_out),real_out)
    fake_loss = cross_entropy(tf.zeros_like(fake_out),fake_out)
    return read_loss + fake_loss
    

In [43]:
def generator_loss(fake_out):
    return cross_entropy(tf.ones_like(fake_out),fake_out)

In [44]:
generator_opt = tf.keras.optimizers.Adam(1e-4)
discriminator_opt = tf.keras.optimizers.Adam(1e-4)

In [45]:
EPOCHS = 100
noise_dim = 100

num_exp_to_generate = 16

seed = tf.random.normal([num_exp_to_generate,noise_dim])

In [46]:
generator = generator_model()

In [47]:
discriminator = discriminator_model()

In [49]:
def train_step(images):
    noise = tf.random.normal([BATCH_SIZE,NOISE_DIM])
    with tf.GradientTape() as gen_tape,tf.GradientTape() as disc_tape:
         
        real_out = discriminator(images,training=True)
        gen_image = generator(noise,training=True)
        fake_out = discriminator(gen_image,training=True)
         
        gen_loss = generator_loss(fake_out)
        disc_loss = discriminator_loss(real_out,fake_out)
        
    gradient_gen = gen_tape.gradient(gen_loss,generator.trainable_variables)
    gradient_disc = disc_tape.gradient(disc_loss,discriminator.trainable_variables)
    
    generator_optimizer.apply_gradients(zip(gradient_gen,generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradient_disc,discriminator.trainable_variables))


IndentationError: unindent does not match any outer indentation level (<tokenize>, line 11)

In [ ]:
def generate_polt_image(gen_model,test_noise):
    pre_images = gen_model(test_noise,training=False)
    fig = plt.figure(figsize=(4,4))
    for i in range(pre_images.shape[0]):
        plt.subplot(4,4,i+1)              
        plt.imshow((pre_images[i, :, :, 0] + 1)/2, cmap='gray')
        plt.axis('off')
    plt.show()

In [ ]:
def train(dataset,epochs):
    for epoch in range(epochs):
        for image_batch in dataset:
            train_step(image_batch)
            print('.',end='')
        genrate_plot_image(generator,seed)

In [ ]:
train(datasets,EPOCHS)